In [170]:
traindata_file = "traindata_x_2.json"
model="vi_core_news_lg"
output_dir='/output'
prefix_name = "saller_xx"
n_iter=30
n_data =15000


In [171]:
import os
import json
import spacy
from pyvi import ViTokenizer, ViPosTagger
from spacy.tokens import DocBin
from tqdm import tqdm
import random


In [172]:
def load_data(file):
    with open(traindata_file, 'r', encoding="utf8") as f:
        return [json.loads(line) for line in f.readlines()]

In [173]:
data = load_data(traindata_file)[0]

In [174]:
data

{'content': '15/05/2021\nMở Bán DỰ ÁN NHÀ Ở XÃ HỘI Thượng Thanh, Long Biên giá từ 16 triệu Miên phí HỒ S\nLoại tin rao Bán căn hộ chung cư\nHà Nội\nGia Quất\nLong Biên\nTên liên hệ Nguyễn Phượng\nGiá 580 Triệu\nGia Quất - Phường Ngọc Lâm -  Long Biên -  Hà Nội\nDiện tích 37\n1\nMặt tiền 4\n0981922984',
 'annotation': [{'label': ['date'],
   'points': [{'start': 0, 'end': 10, 'text': '15/05/2021'}]},
  {'label': ['title'],
   'points': [{'start': 11,
     'end': 90,
     'text': 'Mở Bán DỰ ÁN NHÀ Ở XÃ HỘI Thượng Thanh, Long Biên giá từ 16 triệu Miên phí HỒ S'}]},
  {'label': ['category'],
   'points': [{'start': 91,
     'end': 123,
     'text': 'Loại tin rao Bán căn hộ chung cư'}]},
  {'label': ['region'],
   'points': [{'start': 124, 'end': 131, 'text': 'Hà Nội'}]},
  {'label': ['street'],
   'points': [{'start': 132, 'end': 140, 'text': 'Gia Quất'}]},
  {'label': ['district'],
   'points': [{'start': 141, 'end': 150, 'text': 'Long Biên'}]},
  {'label': ['user'],
   'points': [{'st

In [175]:

def convert_data(traindata_file):
    try:
        training_data = []
        lines = load_data(traindata_file)[0:1000]
        for line in lines:
            data = line

            text = data['content']
            entities = []
            if data['annotation'] is not None:
                for annotation in data['annotation']:
                    # only a single point in text annotation.
                    point = annotation['points'][0]
                    labels = annotation['label']
                    # handle both list of labels or a single label.
                    if not isinstance(labels, list):
                        labels = [labels]

                    for label in labels:
                        # dataturks indices are both inclusive [start, end]
                        # but spacy is not [start, end)
                        check = True
#                         for s, e, l in entities:
#                             if not (point['start'] < point['end'] < s < e or point['start'] > point['end'] > s > e):
#                                 check = False
#                                 break
                                
                        if check and label in ["user"]:      
                            entities.append((
                                point['start'],
                                point['end'], # + 1,
                                label
                            ))

            training_data.append((text, {"entities": entities}))
        return training_data
    except Exception:
#         logging.exception("Unable to process " + traindata_file)
        return None

SPACY_DATA = convert_data(traindata_file)
print(SPACY_DATA[0])

('15/05/2021\nMở Bán DỰ ÁN NHÀ Ở XÃ HỘI Thượng Thanh, Long Biên giá từ 16 triệu Miên phí HỒ S\nLoại tin rao Bán căn hộ chung cư\nHà Nội\nGia Quất\nLong Biên\nTên liên hệ Nguyễn Phượng\nGiá 580 Triệu\nGia Quất - Phường Ngọc Lâm -  Long Biên -  Hà Nội\nDiện tích 37\n1\nMặt tiền 4\n0981922984', {'entities': [(151, 176, 'user')]})


In [176]:

from spacy.training import Example
from spacy.util import minibatch

TRAIN_DATA = SPACY_DATA[0:n_data]

nlp = spacy.blank("vi")
# nlp = spacy.load('model')

examples = []
for text, annots in TRAIN_DATA:
    doc = nlp.make_doc(text)
#     print(doc)
    examples.append(Example.from_dict(doc, annots))
    
# nlp = spacy.blank("vi")

if "ner" not in nlp.pipe_names:
    nlp.add_pipe("ner", last=True)

# ner = nlp.get_pipe("ner")
# for _, annotations in TRAIN_DATA:
#     for ent in annotations.get('entities'):
#         ner.add_label(ent[2])
# move_names = list(ner.move_names)
# print(move_names) 

other_pipes = [pipe for pipe in nlp.pipe_names if pipe != "ner"]


        


In [177]:
with nlp.disable_pipes(*other_pipes):
    nlp.initialize(lambda: examples)
    for i in range(n_iter):
        print("Iter ", i)
        random.shuffle(examples)
        losses={}
        for batch in minibatch(examples, size=8):
            nlp.update(batch,losses=losses)
        print("Losses: ", losses)
        

Iter  0
Losses:  {'ner': 3526.070485155662}
Iter  1
Losses:  {'ner': 60.84074593138466}
Iter  2
Losses:  {'ner': 35.03432810550497}
Iter  3
Losses:  {'ner': 13.949434752498027}
Iter  4
Losses:  {'ner': 11.491632747470918}
Iter  5
Losses:  {'ner': 15.554249184000733}
Iter  6
Losses:  {'ner': 14.885500753347804}
Iter  7
Losses:  {'ner': 2.7855378013278154}
Iter  8
Losses:  {'ner': 6.440996213512129}
Iter  9
Losses:  {'ner': 0.18970157666832135}
Iter  10
Losses:  {'ner': 2.2328461483942355}
Iter  11
Losses:  {'ner': 0.00016730402688322843}
Iter  12
Losses:  {'ner': 6.119175906316626e-07}
Iter  13
Losses:  {'ner': 3.670075585572401e-07}
Iter  14
Losses:  {'ner': 2.6069432311147736e-06}
Iter  15
Losses:  {'ner': 9.711238127676789e-08}
Iter  16
Losses:  {'ner': 3.086598485334493e-08}
Iter  17
Losses:  {'ner': 7.884871178078166e-08}
Iter  18
Losses:  {'ner': 4.0759812612512846e-08}
Iter  19
Losses:  {'ner': 3.0288398225869105e-08}
Iter  20
Losses:  {'ner': 1.2443710494717027e-07}
Iter  21
Los

In [179]:
# nlp.to_disk("./en_example_pipeline")
# test_text = """BÁN GẤP NHÀ PHỐ ĐÀO TẤN, DIỆN TÍCH 50M2, XÂY 4 TẦNG, MẶT TIỀN 5.5M, GIÁ 5.3 Khu Vực: Bán Nhà Riêng Tại Đường Đào Tấn - Phường Cống Vị - Ba Đình - Hà Nội Giá: 5,3 Tỷ  Diện tích: 50 m² hôm nay Thông tin mô tả + Nhà nằm trong ngõ 12 Đào Tấn ô tô tránh có vỉa hè, ngõ thông nhiều phố, nhà cách ô tô đúng 20m, nhà vẫn còn rất mới, khách mua về là vào ở được ngay. + Nhà mới xây được mấy năm nhưng còn rất chắc chắn. Thiết kế nhà bê tông cốt thép chắc chắn, thiết kế hiện đại, nhà bao gồm: phòng khách, bếp, 04 phòng ngủ, phòng thờ, sân phơi + Sổ đỏ chính chủ, pháp lý rõ ràng, sẵn sang giao dịch + Giá 5.3 tỷ, có thương lượng cho khách có thiện trí Liên Hệ: Thanh Tùng BĐS Ba Đình: 0912142902, Quý khách gọi ngay để được tư vấn nhiệt tình và xem nhà miễn phí Cần Bán, ở ngay, ngõ thoáng, nhà còn mới, Ngõ thông, Đào Tấn, Ba Đình Liên hệ: 09121*** hiện số Loại tin rao Bán nhà riêng Pháp lý Sổ Hồng Số phòng 0 Đường vào 0 m Mặt tiền 0 m Số tầng 4 Số toilet 0 Thông tin liên hệ Tên liên hệ: Phan Thanh Tùng Loại tin: Môi giới Địa chỉ: Ba Đình Tỉnh thành Hà Nội Điện thoại: 0912142902 Email: thanhtungks68@gmail.com"""
test_text = """
Loại tin Bán nhà riêng 
Tên liên hệ: Phan Thanh Tùng Tèo Tí
Giá 4,7 Tỷ 
Diện tích 38 m² 
Số tầng 5 
Số phòng 0 
hôm nay 
0912142902 
Mặt tiền 0 m"""
doc = nlp(test_text)
for ent in doc.ents:
    print(" > ", ent.label_, ent.text)


 >  user Tên liên_hệ: Phan_Thanh_Tùng Tèo_Tí


In [180]:

nlp.to_disk(os.path.join("model_saved", "%s_%d_%d"%(prefix_name,n_iter,n_data)))
print("Model saved")

Model saved


In [63]:
import random

def train_spacy(TRAIN_DATA, iterations):

    #Create the blank spacy model
    nlp = spacy.blank("en")
    
    #add the ner component to the pipeline if it's not there
    if "ner" not in nlp.pipe_names:
        ner = nlp.create_pipe("ner")
        nlp.add_pipe(ner, last=True)
    
    #add all labels to the spaCy model
    for _, annotations in TRAIN_DATA:
        for ent in annotations.get("entities"):
            ner.add_label(ent[2])
    
    #eliminate the effect of the training on other pipes and 
    other_pipes = [pipe for pipe in nlp.pipe_names if pipe != "ner"]
    
    #begin training
    with nlp.disable_pipes(*other_pipes):
        optimizer = nlp.begin_training()
        for itn in range(iterations):
            print ("Starting iteration " + str(itn))
            random.shuffle(TRAIN_DATA)
            losses = {}
            for text, annotations in TRAIN_DATA:
                nlp.update(
                            [text],
                            [annotations],
                            drop=0.2,
                            sgd=optimizer,
                            losses=losses
                )
            print
    return (nlp)

#run function and create a trained model
trained_nlp = train_spacy(TRAIN_DATA, 10)

NameError: name 'TRAIN_DATA' is not defined

In [72]:
nlp = spacy.blank("vi")
def create_spacy_data(data): 
    docbin = DocBin()
    for text, annot in tqdm(data):
        doc = nlp.make_doc(text)
        ents = []
        for start, end, label in annot["entities"]:
            span = doc.char_span(start, end, label=label, alignment_mode="contract")
            if span:
                ents.append(span)
            else:
                print("skip")
        doc.ents = ents
        docbin.add(doc)
    return docbin
split_point = int(len(SPACY_DATA)*0.9)
train_data = create_spacy_data(SPACY_DATA[0:split_point])
train_data.to_disk("./data/train_data.spacy")
valid_data = create_spacy_data(SPACY_DATA[split_point:-1])
valid_data.to_disk("./data/valid_data.spacy")

 21%|████████████████▋                                                              | 207/982 [00:00<00:01, 708.82it/s]

skip
skip
skip
skip
skip
skip
skip
skip


100%|███████████████████████████████████████████████████████████████████████████████| 109/109 [00:00<00:00, 768.05it/s]


skip
skip
skip
skip
skip
